In [2]:
# 당일 크롤링
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import time
import re
import mysql.connector

# 크롬 드라이버 설정
service = Service('/path/to/chromedriver')  # 크롬 드라이버 경로 설정
driver = webdriver.Chrome()

# 사이트 접속
url = "https://www.kebhana.com/cms/rate/index.do?contentUrl=/cms/rate/wpfxd651_01i.do#//HanaBank"
driver.get(url)

# 1. 조회일 입력
date_input = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "tmpInqStrDt"))
)
# date_input.clear()
# date_input.send_keys("2024-10-11")

# 2. 조회 버튼 클릭
search_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "a.btnDefault.bg"))
)
search_button.click()

# 3. 5초 대기
time.sleep(5)

# 4. 기준일, 고시회차, 고시시간 데이터 추출
info_span = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "span.fl"))
)
info_text = info_span.text
print(info_text)  # 기준일, 고시회차, 고시시간 출력

# 정규 표현식을 사용하여 기준일, 고시회차, 고시시간 추출
match = re.search(r'기준일\s*:\s*(\d{4}년\d{2}월\d{2}일)고시회차\s*:\s*(\d+)회차고시시간\s*:\s*(\d{2}시\d{2}분\d{2}초)', info_text)

if match:
    reference_date = match.group(1).replace("년", "-").replace("월", "-").replace("일", "")
    announcement_round = int(match.group(2))
    announcement_time = match.group(3).replace("시", ":").replace("분", ":").replace("초", "")
    
    print(f"기준일: {reference_date}")
    print(f"고시회차: {announcement_round}")
    print(f"고시시간: {announcement_time}")
else:
    print("데이터를 추출하지 못했습니다.")

# 5. 테이블 데이터 크롤링
table_body = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "#searchContentDiv > div.printdiv > table > tbody"))
)
table_rows = table_body.find_elements(By.TAG_NAME, "tr")

# 데이터 리스트로 저장
data_to_insert = []
for row in table_rows:
    cells = row.find_elements(By.TAG_NAME, "td")
    row_data = [cell.text for cell in cells]
    if row_data:
        # 각 row_data에 기준일, 고시회차, 고시시간 추가
        row_data.append(reference_date)
        row_data.append(announcement_round)
        row_data.append(announcement_time)
        data_to_insert.append(row_data)

# 브라우저 종료
driver.quit()

# MySQL 연결 정보 설정
config = {
    'user': 'yangjinan',
    'password': 'ALsk85200!!',
    'host': 'hanaex.mysql.database.azure.com',
    'database': 'my_database',
    'port': 3306
}

# MySQL에 데이터 저장
try:
    # MySQL에 연결
    connection = mysql.connector.connect(**config)
    cursor = connection.cursor()

    # SQL 삽입 쿼리
    insert_query = """
    INSERT INTO world_ex_table 
    (currency, cash_buy, cash_buy_spread, cash_sell, cash_sell_spread, remit_send, remit_receive, 
    foreign_check_sell, base_rate, exchange_commission, non_convertible_rate, reference_date, 
    announcement_round, announcement_time)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    # 데이터 삽입
    for row in data_to_insert:
        cursor.execute(insert_query, row)

    # 커밋 (데이터 실제 저장)
    connection.commit()

    print(f"{cursor.rowcount} rows inserted.")

except mysql.connector.Error as err:
    print(f"Error: {err}")

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed.")


고시일시 : 2024년10월11일 12시47분00초 (224회차)
데이터를 추출하지 못했습니다.


NameError: name 'reference_date' is not defined

In [3]:
# 전날꺼 크롤링
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import time
import re
import mysql.connector

# 크롬 드라이버 설정
service = Service('/path/to/chromedriver')  # 크롬 드라이버 경로 설정
driver = webdriver.Chrome()

# 사이트 접속
url = "https://www.kebhana.com/cms/rate/index.do?contentUrl=/cms/rate/wpfxd651_01i.do#//HanaBank"
driver.get(url)

# 1. 조회일 입력
date_input = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "tmpInqStrDt"))
)
date_input.clear()
date_input.send_keys("2024-10-11")
time.sleep(3)
# 2. 조회 버튼 클릭
search_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "a.btnDefault.bg"))
)
search_button.click()

# 3. 5초 대기
time.sleep(5)

# 4. 기준일, 고시회차, 고시시간 데이터 추출
# 4. 기준일, 고시회차, 고시시간 데이터 추출
info_span = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "span.fl"))
)
info_text = info_span.text
print(info_text)  # 기준일, 고시회차, 고시시간 출력

# 정규 표현식을 사용하여 기준일, 고시회차, 고시시간 추출
date_match = re.search(r'(\d{4}년\d{2}월\d{2}일)', info_text)
round_match = re.search(r'\((\d+)회차\)', info_text)
time_match = re.search(r'(\d{2}시\d{2}분\d{2}초)', info_text)

if date_match and round_match and time_match:
    # 기준일 형식 변환
    reference_date = date_match.group(1).replace("년", "-").replace("월", "-").replace("일", "")
    # 고시회차 추출
    announcement_round = int(round_match.group(1))
    # 고시시간 형식 변환
    announcement_time = time_match.group(1).replace("시", ":").replace("분", ":").replace("초", "")
    
    print(f"기준일: {reference_date}")
    print(f"고시회차: {announcement_round}")
    print(f"고시시간: {announcement_time}")
else:
    print("데이터를 추출하지 못했습니다.")

reference_date="2024-10-11"
# 5. 테이블 데이터 크롤링
table_body = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "#searchContentDiv > div.printdiv > table > tbody"))
)
table_rows = table_body.find_elements(By.TAG_NAME, "tr")

# 데이터 리스트로 저장
data_to_insert = []
for row in table_rows:
    cells = row.find_elements(By.TAG_NAME, "td")
    row_data = [cell.text for cell in cells]
    if row_data:
        # 각 row_data에 기준일, 고시회차, 고시시간 추가
        row_data.append(reference_date)
        row_data.append(announcement_round)
        row_data.append(announcement_time)
        data_to_insert.append(row_data)

# 브라우저 종료
driver.quit()

# MySQL 연결 정보 설정
config = {
    'user': 'yangjinan',
    'password': 'ALsk85200!!',
    'host': 'hanaex.mysql.database.azure.com',
    'database': 'my_database',
    'port': 3306
}

# MySQL에 데이터 저장
try:
    # MySQL에 연결
    connection = mysql.connector.connect(**config)
    cursor = connection.cursor()

    # SQL 삽입 쿼리
    insert_query = """
    INSERT INTO world_ex_table 
    (currency, cash_buy, cash_buy_spread, cash_sell, cash_sell_spread, remit_send, remit_receive, 
    foreign_check_sell, base_rate, exchange_commission, non_convertible_rate, reference_date, 
    announcement_round, announcement_time)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    # 데이터 삽입
    for row in data_to_insert:
        cursor.execute(insert_query, row)

    # 커밋 (데이터 실제 저장)
    connection.commit()

    print(f"{cursor.rowcount} rows inserted.")

except mysql.connector.Error as err:
    print(f"Error: {err}")

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed.")


고시일시 : 2024년10월11일 12시47분00초 (224회차)
기준일: 2024-10-11
고시회차: 224
고시시간: 12:47:00
1 rows inserted.
MySQL connection is closed.
